<a href="https://colab.research.google.com/github/stillchen1/Discord_bot/blob/main/Discord_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install discord.py
!pip install nest_asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.9 MB/s eta 0:00:00


In [ ]:
import discord
import asyncio
from discord.ext import commands
from google.colab import userdata
import nest_asyncio
import random
import json
import os

nest_asyncio.apply()

class MyClient(discord.Client):
    async def on_ready(self):
        print(f'Logged on as {self.user}!')

    async def on_message(self, message):
        print(f'Message from {message.author}: {message.content}')

# Setting up intents
intents = discord.Intents.default()
intents.message_content = True

# Instantiate the client with intents
bot = commands.Bot(command_prefix="!", intents=intents)

@bot.command()
async def synccommands(ctx):
  await ctx.bot.tree.sync()
  await ctx.send("同步完成！")

@bot.hybrid_command()
async def ping(ctx):
  """检测机器人是否在线"""
  await ctx.send("pong!")

@bot.hybrid_command()
async def add(ctx, left: int, right: int):
  """两位整数加法运算"""
  await ctx.send(left + right)

class PlayView(discord.ui.View):
    def __init__(self):
        super().__init__()
        self.total_games = 0   # Tracks total games played
        self.player_wins = 0   # Tracks the number of times the player wins
        self.max_games = 5     # Maximum number of games allowed
        self.win_threshold = 3 # Wins required to pass

    def get_content(self, label):
        choices = ["剪刀", "石头", "布"]
        bot_choice = random.choice(choices)

        # Determine the result
        if bot_choice == label:  # Draw
            result = "平局"
        elif (bot_choice == "剪刀" and label == "布") or \
             (bot_choice == "石头" and label == "剪刀") or \
             (bot_choice == "布" and label == "石头"):
            result = "你输了TT"
        else:
            result = "你赢啦！"
            self.player_wins += 1  # Increment player wins if they win

        self.total_games += 1  # Increment total games played

        # Check if the game is over
        if self.total_games >= self.max_games:
            if self.player_wins >= self.win_threshold:
                return f"你出的是{label}, Syde出了{bot_choice}，{result}\n🎉 恭喜你赢了 {self.player_wins}/{self.total_games} 场，你通过了！"
            else:
                return f"你出的是{label}, Syde出了{bot_choice}，{result}\n😞 你赢了 {self.player_wins}/{self.total_games} 场，很遗憾你没有通过。"
        else:
            return f"你出的是{label}, Syde出了{bot_choice}，{result}\n当前战绩: {self.player_wins}/{self.total_games} (需要赢 {self.win_threshold} 场以通过)"

    @discord.ui.button(label="剪刀", style=discord.ButtonStyle.green, emoji="✌️")
    async def scissors(self, interaction: discord.Interaction, button: discord.Button):
        content = self.get_content(button.label)
        if self.total_games >= self.max_games:
            await interaction.response.edit_message(content=content, view=None)  # Disable the view when the game ends
        else:
            await interaction.response.edit_message(content=content)

    @discord.ui.button(label="石头", style=discord.ButtonStyle.green, emoji="✊")
    async def rock(self, interaction: discord.Interaction, button: discord.Button):
        content = self.get_content(button.label)
        if self.total_games >= self.max_games:
            await interaction.response.edit_message(content=content, view=None)  # Disable the view when the game ends
        else:
            await interaction.response.edit_message(content=content)

    @discord.ui.button(label="布", style=discord.ButtonStyle.green, emoji="✋")
    async def paper(self, interaction: discord.Interaction, button: discord.Button):
        content = self.get_content(button.label)
        if self.total_games >= self.max_games:
            await interaction.response.edit_message(content=content, view=None)  # Disable the view when the game ends
        else:
            await interaction.response.edit_message(content=content)

    @discord.ui.button(label="退出", style=discord.ButtonStyle.red)
    async def quit(self, interaction: discord.Interaction, button: discord.Button):
        await interaction.response.edit_message(content="你退出了游戏", view=None)


@bot.hybrid_command()
async def play(ctx):
    """石头剪刀布游戏（非常无聊，5局3胜）"""
    await ctx.send("石头剪刀布，选择你要出什么", view=PlayView())



# 21点 Load or initialize balances

balances = {}  # Initialize balances as an empty dictionary

file_path = "/content/Discord_bot/balances.json"  # Define the file path
if os.path.exists(file_path):  # Check if the file exists
    try:
        with open(file_path, "r") as file:
            balances = json.load(file)
    except json.JSONDecodeError:
        print("Error decoding JSON from balances.json. Using an empty dictionary.")
        # You can add more error handling here if needed

def save_balances():
    """Save the balances to a file."""
    with open(file_path, "w") as file:  # Use the defined file path
        json.dump(balances, file)

def get_balance(user_id):
    """Get a user's balance."""
    return balances.get(str(user_id), 3000)  # Default balance is 3000 coins

def update_balance(user_id, amount):
    """Update a user's balance."""
    balances[str(user_id)] = get_balance(user_id) + amount
    save_balances()

class BlackjackView(discord.ui.View):
    def __init__(self, user_id, amount):
        super().__init__()
        self.deck = self.create_deck()  # Create a shuffled deck
        self.player_hand = []  # Player's cards
        self.dealer_hand = []  # Dealer's cards
        self.user_id = user_id
        self.amount = amount
        self.game_over = False
        self.on_game_end = None  # Callback for game result

        # Deal initial cards
        self.player_hand.append(self.draw_card())
        self.player_hand.append(self.draw_card())
        self.dealer_hand.append(self.draw_card())
        self.dealer_hand.append(self.draw_card())

    def create_deck(self):
        """Create and shuffle a deck of cards."""
        suits = ['♠', '♥', '♦', '♣']
        ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
        deck = [f"{rank}{suit}" for rank in ranks for suit in suits]
        random.shuffle(deck)
        return deck

    def draw_card(self):
        """Draw a card from the deck."""
        return self.deck.pop()

    def calculate_score(self, hand):
        """Calculate the score of a hand."""
        values = {'2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10,
                  'J': 10, 'Q': 10, 'K': 10, 'A': 11}
        score = 0
        ace_count = 0

        for card in hand:
            rank = card[:-1]  # Get the rank (e.g., "10" from "10♠")
            score += values[rank]
            if rank == 'A':
                ace_count += 1

        # Adjust for Aces (make them 1 if score > 21)
        while score > 21 and ace_count:
            score -= 10
            ace_count -= 1

        return score

    def check_winner(self):
        """Determine the winner and update the balance."""
        player_score = self.calculate_score(self.player_hand)
        dealer_score = self.calculate_score(self.dealer_hand)

        if player_score > 21:
            # Player busts
            result = f"你爆牌了！你输了 {self.amount} 金币 😞"
        elif dealer_score > 21:
            # Dealer busts; player wins double the bet
            update_balance(self.user_id, self.amount * 2)
            result = f"庄家爆牌了！你赢了 {self.amount * 2} 金币 🎉"
        elif self.game_over:
            if player_score > dealer_score:
                # Player has a higher score than the dealer
                update_balance(self.user_id, self.amount * 2)
                result = f"你赢了 {self.amount * 2} 金币 🎉"
            elif player_score < dealer_score:
                # Dealer has a higher score than the player
                result = f"你输了 {self.amount} 金币 😞"
            else:
                # It's a tie; refund the bet
                update_balance(self.user_id, self.amount)
                result = "平局！你的下注已返还。"
        else:
            # No winner yet, continue playing
            result = None

        save_balances()
        return result

    def render_hand(self, hand):
        """Render a hand as a string."""
        return ', '.join(hand)

    @discord.ui.button(label="Hit (要牌)", style=discord.ButtonStyle.green)
    async def hit(self, interaction: discord.Interaction, button: discord.Button):
        """Player chooses to draw a card."""
        if self.game_over:
            await interaction.response.send_message("游戏已结束，请重新开始。", ephemeral=True)
            return

        self.player_hand.append(self.draw_card())
        player_score = self.calculate_score(self.player_hand)

        # Check for bust or 21
        if player_score > 21:
            self.game_over = True # Player busts
            winner = self.check_winner() # Check winner only if bust
        elif player_score == 21:
            self.game_over = True # Player gets 21
            winner = self.check_winner() # Check winner only if 21
        else:
            winner = None # No definitive winner yet

        winner = self.check_winner()
        if winner: # If winner is not None (meaning the game is over)
            content = (
                f"你的手牌: {self.render_hand(self.player_hand)} (分数: {player_score})\n"
                f"庄家的手牌: {self.render_hand(self.dealer_hand)}\n\n"
                f"{winner}"
            )
            await interaction.response.edit_message(content=content, view=None)
        else: # If winner is None (meaning the game is not over)
            content = f"你的手牌: {self.render_hand(self.player_hand)} (分数: {player_score})\n庄家的手牌: {self.dealer_hand[0]}, ??"
            await interaction.response.edit_message(content=content)

    @discord.ui.button(label="Stand (停牌)", style=discord.ButtonStyle.blurple)
    async def stand(self, interaction: discord.Interaction, button: discord.Button):
        """Player chooses to stand."""
        self.game_over = True

        # Dealer logic (dealer must hit until score >= 17)
        while self.calculate_score(self.dealer_hand) < 17:
            self.dealer_hand.append(self.draw_card())

        winner = self.check_winner()
        dealer_score = self.calculate_score(self.dealer_hand)
        player_score = self.calculate_score(self.player_hand)

        content = (
            f"你的手牌: {self.render_hand(self.player_hand)} (分数: {player_score})\n"
            f"庄家的手牌: {self.render_hand(self.dealer_hand)} (分数: {dealer_score})\n\n"
            f"{winner}"
        )
        await interaction.response.edit_message(content=content, view=None)

    @discord.ui.button(label="Quit (退出)", style=discord.ButtonStyle.red)
    async def quit(self, interaction: discord.Interaction, button: discord.Button):
        """Quit the game."""
        self.game_over = True
        await interaction.response.edit_message(content="你退出了游戏。", view=None)

@bot.hybrid_command()
async def balance(ctx):
    """查询你的21点游戏余额"""
    user_balance = get_balance(ctx.author.id)
    await ctx.send(f"你的当前余额是 {user_balance} 金币。")

def get_ranked_balances(balances):
       """Sorts balances in descending order and returns a list of (user_id, balance) tuples."""
       sorted_balances = sorted(balances.items(), key=lambda item: item[1], reverse=True)
       return sorted_balances

@bot.hybrid_command()
async def rank(ctx):
        """显示所有玩家的21点金币排行"""
        ranked_balances = get_ranked_balances(balances)  # Get sorted balances

        # Build the message to send
        message = "**21点游戏排行榜**\n"
        for i, (user_id, balance) in enumerate(ranked_balances):
            user = await bot.fetch_user(int(user_id))  # Get user object using user_id
            message += f"{i + 1}. {user.name}: {balance} 金币\n"

        await ctx.send(message)

@bot.hybrid_command()
async def bet(ctx, amount: int):
    """输入下注金额开启一局21点游戏"""
    if amount <= 0:
        await ctx.send("下注金额必须大于 0！")
        return

    user_balance = get_balance(ctx.author.id)
    if amount > user_balance:
        await ctx.send("你的余额不足！")
        return

    update_balance(ctx.author.id, -amount)

    view = BlackjackView(ctx.author.id, amount)
    player_score = view.calculate_score(view.player_hand)
    content = (
        f"你的手牌: {view.render_hand(view.player_hand)} (分数: {player_score})\n"
        f"庄家的手牌: {view.dealer_hand[0]}, ??\n\n"
        f"你下注了 {amount} 金币！"
    )

    await ctx.send(content=content, view=view)

    view.on_game_end = lambda result_message: ctx.send(result_message)


# Run the bot
bot.run(userdata.get('My_TOKEN'))


2024-11-23 16:01:55 INFO     discord.client logging in using static token
2024-11-23 16:01:55 INFO     discord.client logging in using static token
2024-11-23 16:01:55 INFO     discord.client logging in using static token
2024-11-23 16:01:55 INFO     discord.client logging in using static token
2024-11-23 16:01:55 INFO     discord.client logging in using static token
2024-11-23 16:01:55 INFO     discord.client logging in using static token
2024-11-23 16:01:55 INFO     discord.client logging in using static token
2024-11-23 16:01:55 INFO     discord.client logging in using static token
2024-11-23 16:01:55 INFO     discord.client logging in using static token
2024-11-23 16:01:55 INFO     discord.client logging in using static token
2024-11-23 16:01:55 INFO     discord.client logging in using static token
INFO:discord.client:logging in using static token
2024-11-23 16:01:56 INFO     discord.gateway Shard ID None has connected to Gateway (Session ID: ef644d6ff6cce6d5235828d4170bf65a).
2024